Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [1]:
# Vanilla ComfyUI Colab Setup – FIXED for huggingface-hub / transformers conflict (Jan 2026)
# ====================================================================================

from pathlib import Path
import os

OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True    #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True #@param {type:"boolean"}

OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

# Mount Drive only for workflow JSON and outputs (keep runtime fast)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("Drive mounted for workflow & output access only.")

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI $WORKSPACE
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
    !echo -= Updating ComfyUI =-
    !git pull

# Install core deps with **pinned versions** to avoid hub >=1.0 crash
!echo -= Install fixed dependencies =-
!pip install --upgrade pip -q
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -q
!pip install accelerate einops safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3 sentencepiece soundfile kornia>=0.7.1 spandrel av sageattention -q

# Critical pins: transformers recent but compatible + hub <1.0
!pip install "transformers>=4.45.0,<4.57.0" "huggingface-hub>=0.23.0,<1.0" -q

# Optional: hf_transfer for faster model downloads later (compatible with hub<1.0)
!pip install hf_transfer -q

# ComfyUI-Manager
if OPTIONS['USE_COMFYUI_MANAGER']:
    %cd custom_nodes
    ![ ! -d ComfyUI-Manager ] && git clone https://github.com/ltdrdata/ComfyUI-Manager
    %cd ComfyUI-Manager
    !git pull
    %cd $WORKSPACE

# Restore custom node deps if needed
if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
    !echo -= Install custom nodes dependencies =-
    !pip install GitPython -q
    !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies
    !pip install comfyui-workflow-templates comfyui-embedded-docs --upgrade --quiet

print("\nVanilla setup finished with compatible versions!")
print("→ Restart runtime NOW (Runtime → Restart runtime)")
print("→ Then run the launch cell below")

Mounted at /content/drive
Drive mounted for workflow & output access only.
-= Initial setup ComfyUI =-
Cloning into '/content/ComfyUI'...
remote: Enumerating objects: 29184, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 29184 (delta 17), reused 8 (delta 8), pack-reused 29154 (from 2)
Receiving objects: 100% (29184/29184), 69.64 MiB | 13.86 MiB/s, done.
Resolving deltas: 100% (19787/19787), done.
/content/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install fixed dependencies =-
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.3 MB/s eta 0:00:00
/content/ComfyUI/custom_nodes
Cloning into 'ComfyUI-Manager'...
remote: Enumerating objects: 27813, done.
remote: Counting objects: 100% (517/517), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 27813 (delta 410), reused 283 (delta 271), pack-reused 27296 (from 4)
Receiving objects: 100% (27813/27813), 124.05 MiB | 15.81 MiB/s, done.
Re

# fixes

In [2]:
# Cell: Install PyAV (av) - fixes the import error for video_types.py
!pip install av

# Install torchsde - required by ComfyUI k_diffusion sampling
!pip install torchsde

# Install comfy_kitchen for FP8/FP4 support
!pip install comfy-kitchen --quiet

# Force-fix: Re-pin transformers + huggingface-hub + tokenizers after everything else
# This overrides any late upgrades from Manager or restore-dependencies

!pip uninstall -y huggingface-hub transformers tokenizers -q
!pip install "huggingface-hub==0.36.0" "transformers==4.55.2" "tokenizers==0.21.0" --no-deps -q
# --no-deps prevents pulling newer versions again

# Optional: keep hf_transfer if you want fast downloads later (it works with 0.36.0)
!pip install hf_transfer -q

# Verify versions
import importlib.metadata as metadata
print("Forced versions:")
for pkg in ["huggingface-hub", "transformers", "tokenizers"]:
    try:
        v = metadata.version(pkg)
        print(f"Name: {pkg}\nVersion: {v}")
    except:
        print(f"{pkg}: not found")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torchsde]
Forced versions:
Name: huggingface-hub
Version: 0.36.0
Name: transformers
Version: 4.55.2
Name: tokenizers
Version: 0.21.0


# Custom Models

In [3]:
# Download ALL required LTX-2 models

import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

print("📥 Downloading LTX-2 models...\n")

# 1. Main checkpoint (FP8 version)
print("1️⃣ Downloading checkpoint...")
!hf download Lightricks/LTX-2 ltx-2-19b-dev-fp8.safetensors \
  --local-dir /content/ComfyUI/models/checkpoints

# 2. Text encoder (download with nested structure)
print("\n2️⃣ Downloading text encoder...")
!hf download Comfy-Org/ltx-2 split_files/text_encoders/gemma_3_12B_it.safetensors \
  --local-dir /content/ComfyUI/models/text_encoders

# Create symlink for flat path (workflow expects both)
source = "/content/ComfyUI/models/text_encoders/split_files/text_encoders/gemma_3_12B_it.safetensors"
target = "/content/ComfyUI/models/text_encoders/gemma_3_12B_it.safetensors"
if os.path.exists(source) and not os.path.exists(target):
    os.symlink(source, target)
    print(f"   ✅ Created symlink for flat path access")

# 3. LoRA
print("\n3️⃣ Downloading LoRA...")
!hf download Lightricks/LTX-2 ltx-2-19b-distilled-lora-384.safetensors \
  --local-dir /content/ComfyUI/models/loras

# 4. Spatial upscaler
print("\n4️⃣ Downloading upscaler...")
!hf download Lightricks/LTX-2 ltx-2-spatial-upscaler-x2-1.0.safetensors \
  --local-dir /content/ComfyUI/models/latent_upscale_models

# Verify all files exist
print("\n" + "="*60)
print("📁 VERIFICATION")
print("="*60)

files_to_check = {
    "Checkpoint": "/content/ComfyUI/models/checkpoints/ltx-2-19b-dev-fp8.safetensors",
    "Text Encoder (nested)": "/content/ComfyUI/models/text_encoders/split_files/text_encoders/gemma_3_12B_it.safetensors",
    "Text Encoder (flat)": "/content/ComfyUI/models/text_encoders/gemma_3_12B_it.safetensors",
    "LoRA": "/content/ComfyUI/models/loras/ltx-2-19b-distilled-lora-384.safetensors",
    "Upscaler": "/content/ComfyUI/models/latent_upscale_models/ltx-2-spatial-upscaler-x2-1.0.safetensors"
}

all_good = True
for name, path in files_to_check.items():
    if os.path.exists(path):
        size_gb = os.path.getsize(path) / (1024**3)
        print(f"✅ {name}: {size_gb:.2f} GB")
    else:
        print(f"❌ {name}: MISSING")
        all_good = False

if all_good:
    print("\n🎉 All models downloaded successfully!")
else:
    print("\n⚠️ Some models are missing. Please check the errors above.")

📥 Downloading LTX-2 models...

1️⃣ Downloading checkpoint...
ltx-2-19b-dev-fp8.safetensors: 100% 27.1G/27.1G [01:21<00:00, 334MB/s] 
Download complete. Moving file to /content/ComfyUI/models/checkpoints/ltx-2-19b-dev-fp8.safetensors
/content/ComfyUI/models/checkpoints/ltx-2-19b-dev-fp8.safetensors

2️⃣ Downloading text encoder...
split_files/text_encoders/gemma_3_12B_it(…): 100% 24.4G/24.4G [01:00<00:00, 404MB/s]
Download complete. Moving file to /content/ComfyUI/models/text_encoders/split_files/text_encoders/gemma_3_12B_it.safetensors
/content/ComfyUI/models/text_encoders/split_files/text_encoders/gemma_3_12B_it.safetensors
   ✅ Created symlink for flat path access

3️⃣ Downloading LoRA...
ltx-2-19b-distilled-lora-384.safetensors: 100% 7.67G/7.67G [00:31<00:00, 247MB/s] 
Download complete. Moving file to /content/ComfyUI/models/loras/ltx-2-19b-distilled-lora-384.safetensors
/content/ComfyUI/models/loras/ltx-2-19b-distilled-lora-384.safetensors

4️⃣ Downloading upscaler...
ltx-2-spatia

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [5]:
import threading
import time
import socket
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

[START] Security scan
[ComfyUI-Manager] Using `uv` as Python module for pip operations.
Using Python 3.12.12 environment at: /usr
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-01-12 09:05:49.842
** Platform: Linux
** Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/ComfyUI
** ComfyUI Base Folder Path: /content/ComfyUI
** User directory: /content/ComfyUI/user
** ComfyUI-Manager config path: /content/ComfyUI/user/__manager/config.ini
** Log path: /content/ComfyUI/user/comfyui.log
Using Python 3.12.12 environment at: /usr
Using Python 3.12.12 environment at: /usr

Prestartup times for custom nodes:
   0.7 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager

Checkpoint files will always be loaded safely.
Total VRAM 81222 MB, total RAM 171061 MB
pytorch version: 2.9.0+cu126
Set vram state to: NORMAL_VRAM
Device: cuda:0 NVIDIA A100-SXM4-80GB : cudaMallo

<IPython.core.display.Javascript object>

to open it in a window you can open this link here:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

[DEPRECATION WARNING] Detected import of deprecated legacy API: /scripts/ui.js. This is likely caused by a custom node extension using outdated APIs. Please update your extensions or contact the extension author for an updated version.
[DEPRECATION WARNING] Detected import of deprecated legacy API: /extensions/core/groupNode.js. This is likely caused by a custom node extension using outdated APIs. Please update your extensions or contact the extension author for an updated version.
[DEPRECATION WARNING] Detected import of deprecated legacy API: /scripts/ui/components/buttonGroup.js. This is likely caused by a custom node extension using outdated APIs. Please update your extensions or contact the extension author for an updated version.
[DEPRECATION WARNING] Detected import of deprecated legacy API: /scripts/ui/components/button.js. This is likely caused by a custom node extension using outdated APIs. Please update your extensions or contact the extension author for an updated version.
